# SETUP

## imports

In [1]:
import string
import numpy as np
from itertools import cycle
import requests
import collections
from collections import deque
from pprint import pprint
import operator
from time import sleep
import itertools
import re
from functools import reduce
from dataclasses import dataclass
from copy import deepcopy
import networkx as nx
import matplotlib.pyplot as plt

## constants

In [2]:
lowercase = string.ascii_lowercase
uppercase = string.ascii_uppercase

## helpers

In [3]:
def get_level_input(lvl_num):
    with open(f"advent_inputs/{lvl_num}.txt") as f:
        level_input=f.read()
        return level_input
    
def print_result(answer):
    pprint("RESULT: "+str(answer))
    print()
    pprint("TIME"+"."*60)
    
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

## INTCODE COMPUTER SPECS

In [18]:
ADD = 1
MULT = 2
INPUT = 3
OUTPUT = 4
JUMP_IF_TRUE = 5
JUMP_IF_FALSE = 6 
LESS_THAN = 7 
EQUALS = 8

END = 99

operation_length = {
    ADD: 4, 
    MULT: 4, 
    INPUT: 2, 
    OUTPUT: 2, 
    END: 1, 
    JUMP_IF_TRUE: 3, 
    JUMP_IF_FALSE: 3, 
    LESS_THAN: 4, 
    EQUALS: 4
}

operation_num_inputs = {
    ADD: 2, 
    MULT: 2, 
    INPUT: 0, 
    OUTPUT: 0, 
    JUMP_IF_TRUE: 2, 
    JUMP_IF_FALSE: 2, 
    LESS_THAN: 2, 
    EQUALS: 2
}

JUMPS = { JUMP_IF_TRUE, JUMP_IF_FALSE }

comp_string = get_level_input("05")
comp_string = list(map(int, comp_string.split(',')))

def print_computer(comp_string):
    for i in range(0, len(comp_string), 10):
        print(f"[{i}]    {comp_string[i:i+10]}")
 
def print_step(win, inputs, output):
    print(f"win: {win}")
    print(f"inputs: {inputs}")    
    print(f"output: {output}")

# 1

## setup

In [5]:
module_weights = get_level_input("01").splitlines()
module_weights = list(map(int, module_weights))

## part one

In [6]:
%%time
total_weight = sum([x//3 - 2 for x in module_weights])
print_result(total_weight)

'RESULT: 3432671'

'TIME............................................................'
CPU times: user 421 µs, sys: 105 µs, total: 526 µs
Wall time: 472 µs


## part two

In [67]:
%%time

def weight_of_weight_gen(m_weight):
    while m_weight>0:
        yield m_weight
        m_weight = m_weight//3 - 2
    
def get_req_fuel(m_weight): 
    return sum([i for i in weight_of_weight_gen(m_weight)][1:])
    
total_weight = sum([get_req_fuel(x) for x in module_weights])
print_result(total_weight)

'RESULT: 5146132'

'TIME............................................................'
CPU times: user 1.14 ms, sys: 284 µs, total: 1.42 ms
Wall time: 1.19 ms


# 2

## setup

In [124]:
OPCODE = 0
INPUT_1 = 1
INPUT_2 = 2
OUTPUT = 3
WINDOW = 4

comp_string = get_level_input("02")
comp_string = list(map(int, comp_string.split(',')))
# comp_string = [1,9,10,3,2,3,11,0,99,30,40,50]


## part one

In [125]:
%%time

def run_computer(comp_string):
    comp_string[1] = 12
    comp_string[2] = 2
    comp_len = len(comp_string)
    for i in range(0, len(comp_string), 4):
        win = comp_string[i:i+WINDOW]
        if(win[OPCODE] == END): return comp_string
        assert(all(x < comp_len for x in win))
        comp_string[win[OUTPUT]] = {
            ADD: comp_string[win[INPUT_1]]+comp_string[win[INPUT_2]],
            MULT: comp_string[win[INPUT_1]]*comp_string[win[INPUT_2]]
        }[win[OPCODE]]
    
print_result(run_computer(list(comp_string))[0])

'RESULT: 5290681'

'TIME............................................................'
CPU times: user 425 µs, sys: 284 µs, total: 709 µs
Wall time: 464 µs


## part two 

In [188]:
%%time

TARGET = 19690720

def run_computer(comp_string, noun, verb):
    comp_string[1] = noun 
    comp_string[2] = verb 
    comp_len = len(comp_string)
    for i in range(0, len(comp_string), 4):
        win = comp_string[i:i+WINDOW]
        if(win[OPCODE] == END): return comp_string[0]
        if(not all(x < comp_len for x in win)): return -1
        comp_string[win[OUTPUT]] = {
            ADD: comp_string[win[INPUT_1]]+comp_string[win[INPUT_2]],
            MULT: comp_string[win[INPUT_1]]*comp_string[win[INPUT_2]]
        }[win[OPCODE]]
    
def find_noun_verb(comp_string):
    for i in range(100):
        for j in range(100):
            res = run_computer(list(comp_string), i, j) 
            if res == -1: continue 
            if(res == TARGET): return i,j

noun, verb = find_noun_verb(list(comp_string))

print_result(100 * noun + verb)

'RESULT: 5741'

'TIME............................................................'
CPU times: user 295 ms, sys: 2.75 ms, total: 298 ms
Wall time: 298 ms


# 3

## setup

In [87]:
wires_input = get_level_input("03").splitlines()
wires_input = [x.split(',') for x in wires_input]

X = 1
Y = 0
CENTER_POINT = (0,0)

def md(p1, p2):
    return (abs(p1[X]-p2[X]) + abs(p1[Y]-p2[Y]))

## part one

In [88]:
%%time

def generate_wire(instructions):
    wire = [CENTER_POINT]
    for x in instructions:
        cur_point = wire[-1]
        wire += {
         'U': [(cur_point[Y]-i, cur_point[X]) for i in range(1, int(x[1:])+1)],
         'D': [(cur_point[Y]+i, cur_point[X]) for i in range(1, int(x[1:])+1)],
         'L': [(cur_point[Y], cur_point[X]-i) for i in range(1, int(x[1:])+1)],
         'R': [(cur_point[Y], cur_point[X]+i) for i in range(1, int(x[1:])+1)]
        }[x[0]]
    return wire
                                  
    
wires = [set(generate_wire(w_i))-{CENTER_POINT} for w_i in wires_input]
res = min([md(x, CENTER_POINT) for x in set.intersection(*wires)])
print_result(res)

'RESULT: 225'

'TIME............................................................'
CPU times: user 271 ms, sys: 5.11 ms, total: 276 ms
Wall time: 276 ms


## part two

In [142]:
%%time

wires = [generate_wire(w_i) for w_i in wires_input]
intersections = set.intersection(*[set(wire) for wire in wires])-{CENTER_POINT}
res = min([wires[0].index(x)+wires[1].index(x) for x in intersections])
print_result(res)

'RESULT: 35194'

'TIME............................................................'
CPU times: user 358 ms, sys: 4.02 ms, total: 362 ms
Wall time: 362 ms


In [ ]:
# 4

In [ ]:
## setup

In [140]:
r_low, r_high = map(int, get_level_input("04").split("-"))

def check_value(val, functions):
    num_list = [int(i) for i in list(str(val))]
    return (all(f(num_list) for f in functions))

## part one

In [143]:
%%time

def check_increase(l):
    return all(l[i] <= l[i+1] for i in range(len(l)-1))

def check_double(l):
    return any(l[i] == l[i+1] for i in range(len(l)-1))

num_poss_pass = 0
for num in range(r_low, r_high+1):
    if(check_value(num, [check_increase, check_double])): num_poss_pass+=1

print_result(num_poss_pass)

'RESULT: 1694'

'TIME............................................................'
CPU times: user 1.67 s, sys: 4.64 ms, total: 1.67 s
Wall time: 1.67 s


## part two 

In [144]:
%%time

def check_increase(l):
    return all(l[i] <= l[i+1] for i in range(len(l)-1))

def check_ungrouped_double(l):
    l = [-1] + l +[-1]
    return any(l[i-1] != l[i] == l[i+1] != l[i+2] for i in range(1, len(l)-1))

num_poss_pass = 0
for num in range(r_low, r_high+1):
    num_list = [int(i) for i in list(str(num))]
    if(check_value(num, [check_increase, check_ungrouped_double])): num_poss_pass += 1

print_result(num_poss_pass)

'RESULT: 1148'

'TIME............................................................'
CPU times: user 2.58 s, sys: 5.62 ms, total: 2.58 s
Wall time: 2.58 s


# 5

## setup

In [135]:
comp_string = get_level_input("05")
comp_string = list(map(int, comp_string.split(',')))

## part one

In [19]:
%%time

def run_computer(comp_string):
    i=0
    last_output = 0
    # print_computer(comp_string)
    while(True):
        opcode_and_settings = "0000"+str(comp_string[i])
        opcode = int(opcode_and_settings[-2:])
        if(opcode == END): return comp_string, last_output
        input_modes = [int(opcode_and_settings[-3]), int(opcode_and_settings[-4]), int(opcode_and_settings[-5])]
        
        window = operation_length[opcode]
        win = comp_string[i: i+window]
        i+=window
        
        output = win[-1]
        num_inputs = operation_num_inputs[opcode]
        inputs = [win[i+1] if input_modes[i] else comp_string[win[i+1]] for i in range(operation_num_inputs[opcode])]
        if(opcode == INPUT): 
            inputs = [int(input("provide input value: ")), 0]
            print("")
        print_step(win,inputs,output)
        if(opcode == OUTPUT): 
            print(f"OUTPUT: {comp_string[output]}\n")
            last_output = output
            continue

        comp_string[output] = {
            ADD: (inputs[0] + inputs[1]),
            MULT: (inputs[0] * inputs[1]),
            INPUT: inputs[0],
        }[opcode]
        # print_computer(comp_string)

print_result(comp_string[run_computer(list(comp_string))[1]])

provide input value:  1



win: [3, 225]
inputs: [1, 0]
output: 225
win: [1, 225, 6, 6]
inputs: [1, 1100]
output: 6
win: [1101, 1, 238, 225]
inputs: [1, 238]
output: 225
win: [104, 0]
inputs: []
output: 0
OUTPUT: 3

win: [1101, 86, 8, 225]
inputs: [86, 8]
output: 225
win: [1101, 82, 69, 225]
inputs: [82, 69]
output: 225
win: [101, 36, 65, 224]
inputs: [36, 70]
output: 224
win: [1001, 224, -106, 224]
inputs: [106, -106]
output: 224
win: [4, 224]
inputs: []
output: 224
OUTPUT: 0

win: [1002, 223, 8, 223]
inputs: [0, 8]
output: 223
win: [1001, 224, 5, 224]
inputs: [0, 5]
output: 224
win: [1, 223, 224, 223]
inputs: [0, 5]
output: 223
win: [102, 52, 148, 224]
inputs: [52, 22]
output: 224
win: [101, -1144, 224, 224]
inputs: [-1144, 1144]
output: 224
win: [4, 224]
inputs: []
output: 224
OUTPUT: 0

win: [1002, 223, 8, 223]
inputs: [5, 8]
output: 223
win: [101, 1, 224, 224]
inputs: [1, 0]
output: 224
win: [1, 224, 223, 223]
inputs: [1, 40]
output: 223
win: [1102, 70, 45, 225]
inputs: [70, 45]
output: 225
win: [1002, 143

In [17]:
%%time
def run_computer(comp_string):
    i=0
    last_output = 0
    # print_computer(comp_string)
    while(True):
        opcode_and_settings = "0000"+str(comp_string[i])
        opcode = int(opcode_and_settings[-2:])
        if(opcode == END): return comp_string, last_output
        input_modes = [int(opcode_and_settings[-3]), int(opcode_and_settings[-4]), int(opcode_and_settings[-5])]
        
        window = operation_length[opcode]
        win = comp_string[i: i+window]
        i+=window
        
        
        output = win[-1]
        num_inputs = operation_num_inputs[opcode]
        inputs = [win[i+1] if input_modes[i] else comp_string[win[i+1]] for i in range(operation_num_inputs[opcode])]
         
        
        if(opcode == INPUT): 
            inputs = [int(input("provide input value: ")), 0]
            print("")
        
        if(opcode == OUTPUT): 
            print(f"OUTPUT: {comp_string[output]}\n")
            last_output = comp_string[output]
            continue
            
        if(opcode in JUMPS):
            i = {
                JUMP_IF_TRUE: inputs[1] if inputs[0] != 0 else i,
                JUMP_IF_FALSE: inputs[1] if inputs[0] == 0 else i, 
            }[opcode]
            
        else:
            comp_string[output] = {
                ADD: (inputs[0] + inputs[1]),
                MULT: (inputs[0] * inputs[1]),
                INPUT: inputs[0],
                LESS_THAN: 1 if inputs[0] < inputs[1] else 0, 
                EQUALS: 1 if inputs[0] == inputs[1] else 0 
            }[opcode]
            # print_computer(comp_string)

print_result(run_computer(list(comp_string))[1])

provide input value:  1



window: 3
inputs: [1, 0]
output: 225
window: 1
inputs: [1, 1100]
output: 6
window: 1
inputs: [1, 238]
output: 225
window: 4
inputs: []
output: 0
OUTPUT: 3

window: 1
inputs: [86, 8]
output: 225
window: 1
inputs: [82, 69]
output: 225
window: 1
inputs: [36, 70]
output: 224
window: 1
inputs: [106, -106]
output: 224
window: 4
inputs: []
output: 224
OUTPUT: 0

window: 2
inputs: [0, 8]
output: 223
window: 1
inputs: [0, 5]
output: 224
window: 1
inputs: [0, 5]
output: 223
window: 2
inputs: [52, 22]
output: 224
window: 1
inputs: [-1144, 1144]
output: 224
window: 4
inputs: []
output: 224
OUTPUT: 0

window: 2
inputs: [5, 8]
output: 223
window: 1
inputs: [1, 0]
output: 224
window: 1
inputs: [1, 40]
output: 223
window: 2
inputs: [70, 45]
output: 225
window: 2
inputs: [28, 48]
output: 224
window: 1
inputs: [1344, -1344]
output: 224
window: 4
inputs: []
output: 224
OUTPUT: 0

window: 2
inputs: [8, 41]
output: 223
window: 1
inputs: [7, 0]
output: 224
window: 1
inputs: [328, 7]
output: 223
window: 1
i

# 6

## setup 

In [115]:
orbits = get_level_input("06").splitlines()
orbits = [orbit.split(')') for orbit in orbits]

## part one

In [113]:
%%time

orbit_graph = nx.DiGraph()
for orbit in orbits:
    orbit_graph.add_edge(*orbit)

orbit_checksum = sum([len(nx.descendants(orbit_graph, node)) for node in orbit_graph.nodes()])

print_result(orbit_checksum)

'RESULT: 144909'

'TIME............................................................'
CPU times: user 448 ms, sys: 6.68 ms, total: 455 ms
Wall time: 460 ms


## part two 

In [117]:
TARGET = "SAN"
START = "YOU"

orbit_graph = nx.Graph()
for orbit in orbits:
    orbit_graph.add_edge(*orbit)

START_SR = list(orbit_graph.neighbors(START))[0]
SAN_SR = list(orbit_graph.neighbors(TARGET))[0]

print_result(nx.shortest_path_length(orbit_graph, str(START_SR), str(SAN_SR)))
    

'RESULT: 259'

'TIME............................................................'


# 7

## setup 

In [44]:
amp_prog = get_level_input("07")
amp_prog = list(map(int, amp_prog.split(',')))
#amp_prog = [3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0] #[3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]
NUM_AMPS = 5

def run_computer(comp_string, c_i):
    i=0
    last_output = 0
    while(True):
        opcode_and_settings = "0000"+str(comp_string[i])
        opcode = int(opcode_and_settings[-2:])
        if(opcode == END): return comp_string, last_output
        input_modes = [int(opcode_and_settings[-3]), int(opcode_and_settings[-4]), int(opcode_and_settings[-5])]
        
        window = operation_length[opcode]
        win = comp_string[i: i+window]
        i+=window        
        output = win[-1]
        num_inputs = operation_num_inputs[opcode]
        inputs = [win[j+1] if input_modes[j] else comp_string[win[j+1]] for j in range(operation_num_inputs[opcode])]
        
        if(opcode == INPUT): 
            inputs = [c_i.pop(0), 0]
            
        if(opcode == OUTPUT): 
            return comp_string[output]
            continue
            
        if(opcode in JUMPS):
            i = {
                JUMP_IF_TRUE: inputs[1] if inputs[0] != 0 else i,
                JUMP_IF_FALSE: inputs[1] if inputs[0] == 0 else i, 
            }[opcode]
            
        else:
            comp_string[output] = {
                ADD: (inputs[0] + inputs[1]),
                MULT: (inputs[0] * inputs[1]),
                INPUT: inputs[0],
                LESS_THAN: 1 if inputs[0] < inputs[1] else 0, 
                EQUALS: 1 if inputs[0] == inputs[1] else 0 
            }[opcode]

## part one

In [15]:
%%time

amp_pos = list(itertools.permutations(list(range(NUM_AMPS))))

max_val = 0

for perm in amp_pos:
    inputs = [perm[0], 0]
    for i in range(NUM_AMPS):
        inputs = [perm[i+1] if i+1<NUM_AMPS else 0, run_computer(amp_prog[:], inputs)]
        if(inputs[1]>max_val):
            max_val = inputs[1]
        
print_result(max_val)

'RESULT: 21760'

'TIME............................................................'
CPU times: user 25.1 ms, sys: 1.66 ms, total: 26.8 ms
Wall time: 26.1 ms


In [45]:
%%time

#amp_prog = [3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]

def run_computer(ip, comp_string, c_i):
    while(True):
        opcode_and_settings = "0000"+str(comp_string[ip])
        opcode = int(opcode_and_settings[-2:])
        if(opcode == END): 
            return -1, ip, c_i
        
        input_modes = [int(opcode_and_settings[-3]), int(opcode_and_settings[-4]), int(opcode_and_settings[-5])]
        
        window = operation_length[opcode]
        win = comp_string[ip: ip+window]
        ip += window
        
        output = win[-1]
        num_inputs = operation_num_inputs[opcode]
        inputs_prog = [win[i+1] if input_modes[i] else comp_string[win[i+1]] for i in range(operation_num_inputs[opcode])] 
        if(opcode == INPUT): 
            inputs_prog = [c_i.pop(0), 0]
        if(opcode == OUTPUT): 
            return comp_string[output], ip, c_i
            
        if(opcode in JUMPS):
            ip = {
                JUMP_IF_TRUE: inputs_prog[1] if inputs_prog[0] != 0 else ip,
                JUMP_IF_FALSE: inputs_prog[1] if inputs_prog[0] == 0 else ip, 
            }[opcode]
        else:
            comp_string[output] = {
                ADD: (inputs_prog[0] + inputs_prog[1]),
                MULT: (inputs_prog[0] * inputs_prog[1]),
                INPUT: inputs_prog[0],
                LESS_THAN: 1 if inputs_prog[0] < inputs_prog[1] else 0, 
                EQUALS: 1 if inputs_prog[0] == inputs_prog[1] else 0 
            }[opcode]

amp_pos = list(itertools.permutations(list(range(NUM_AMPS, NUM_AMPS+5))))

signals = []
for perm in amp_pos:
    i = 0
    inputs = [[perm[j]] for j in range(NUM_AMPS)]
    inputs[0].append(0)
    last_e = 0
    amp_progs = [deepcopy(amp_prog) for j in range(NUM_AMPS)]
    ips = [0 for j in range(NUM_AMPS)]
    while(True):
        output, ips[i%NUM_AMPS],inputs[i%NUM_AMPS] = run_computer(ips[i%NUM_AMPS], amp_progs[i%NUM_AMPS], inputs[i%NUM_AMPS])
        if output == -1:
            break
        if(i%NUM_AMPS == 4):
            last_e = output
        i+=1
        inputs[i%NUM_AMPS].append(output)
    signals.append(last_e)
        
print_result(max(signals))

'RESULT: 69816958'

'TIME............................................................'
CPU times: user 273 ms, sys: 3.63 ms, total: 276 ms
Wall time: 278 ms


# 8

## setup 

In [46]:
orbits = get_level_input("08").splitlines()
orbits = [orbit.split(')') for orbit in orbits]

## part one

## part two